In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [48]:
df = pd.read_csv('train.csv')

In [49]:
df.shape

(404290, 6)

In [50]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [51]:
new_df = df.sample(30000)
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [52]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
163561,How are Fiber One jeans?,Do you like Levis jeans?
212208,What's it like to be addicted to cocaine or cr...,How much/how often does one have to use cocain...
365376,How can changing 500 and 1000 rupee notes end ...,How is India fighting corruption by scrapping ...
300959,What salary I can expect after having 3 years ...,What salary I can expect after having 2 years ...
112365,Why do companies use recruiters rather than fi...,"How do recruiters get paid, and how much?"


In [53]:
from sklearn.feature_extraction.text import CountVectorizer
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv= CountVectorizer(max_features = 3000)
q1_arr,q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [54]:
temp_df1 = pd.DataFrame(q1_arr,index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr,index=ques_df.index)
temp_df  = pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

(30000, 6000)

In [55]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [56]:
temp_df = temp_df.loc[:, ~temp_df.columns.duplicated()]

In [57]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
163561,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
365376,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
300959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,test_size=0.2,random_state=16)
from sklearn.model_selection import train_test_split

# Assuming 'temp_df' is your DataFrame
# The line below is corrected
x_train, x_test, y_train, y_test = train_test_split(temp_df.iloc[:, 0:-1], 
                                                    temp_df.iloc[:, -1], 
                                                    test_size=0.2, 
                                                    random_state=2)

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
accuracy_score(y_test,y_pred)

0.7225

In [60]:
print(temp_df)

        0  1  2  3  4  5  6  7  8  9  ...  2991  2992  2993  2994  2995  2996  \
163561  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
212208  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
365376  0  0  0  1  0  0  0  0  0  0  ...     0     0     0     0     0     0   
300959  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
112365  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
...    .. .. .. .. .. .. .. .. .. ..  ...   ...   ...   ...   ...   ...   ...   
374515  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
16536   0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
346315  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
47064   1  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
28105   0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   

        2997  2998  2999  i

In [61]:
# from xgboost import XGBClassifier
# xgb = XGBClassifier()
# xgb.fit(x_train,y_train)
from xgboost import XGBClassifier
# Assuming x_train and y_train are already defined
xgb = XGBClassifier()
xgb.fit(x_train,y_train)
y_pred = xgb.predict(x_test)
accuracy_score(y_test,y_pred)

0.7143333333333334

In [62]:
assert x_train.columns.is_unique, "Duplicate columns found in x_train!" 

In [63]:
x_train.column

AttributeError: 'DataFrame' object has no attribute 'column'

In [ ]:
##  added new feture   ##

In [ ]:
x